In [1]:
from google.colab import files
uploaded = files.upload()

Saving GOOG.csv to GOOG.csv


# Google Stock Price Prediction using LSTM

Predicting Google's closing stock price using a stacked LSTM deep learning model.

Dataset: Google (GOOG) historical stock data from Yahoo Finance  
Model: Stacked LSTM with Dropout regularization  
Look-back window: 60 days

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

print("TensorFlow version:", tf.__version__)

In [ ]:
df = pd.read_csv('GOOG.csv')
print(df.shape)
df.head()

# Closing Price History
Visualizing raw closing price to understand the overall trend before modelling.

In [ ]:
plt.figure(figsize=(16, 6))
plt.style.use('fivethirtyeight')
plt.title('Google Closing Stock Price', fontsize=16)
plt.plot(df['Close'])
plt.xlabel('Date', fontsize=13)
plt.ylabel('Close Price (USD)', fontsize=13)
plt.tight_layout()
plt.savefig('closing_price.png', dpi=150)
plt.show()

## Data Preparation
LSTM requires MinMax scaling to [0,1] and 60-day sliding window sequences.

In [ ]:
data = df['Close'].values.reshape(-1, 1)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size - 60:]

print(f"Training samples: {train_size}")
print(f"Test samples: {len(scaled_data) - train_size}")

## Building 60-Day Look-back Sequences
For each prediction, model looks at previous 60 closing prices.
X = [day1..day60], y = day61

In [ ]:
def create_sequences(data, window=60):
    X, y = [], []
    for i in range(window, len(data)):
        X.append(data[i-window:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_data)
X_test, y_test = create_sequences(test_data)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

## Model Architecture
Stacked LSTM with Dropout regularization.
- 2 LSTM layers (50 units each)
- Dropout 0.2 after each layer to prevent overfitting
- Dense output layer

In [ ]:
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

## Training
20 epochs, batch size 32. Validation split 10% to monitor overfitting.

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

## Predictions on Test Set

In [ ]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

## Evaluation

In [ ]:
rmse = math.sqrt(mean_squared_error(y_test_actual, predictions))
print(f"RMSE: {rmse:.2f}")

## Predicted vs Actual

In [ ]:
plt.figure(figsize=(16, 6))
plt.style.use('fivethirtyeight')
plt.title('LSTM — Predicted vs Actual Google Stock Price', fontsize=16)
plt.plot(y_test_actual, label='Actual Price', linewidth=1.5)
plt.plot(predictions, label='Predicted Price', linewidth=1.5, linestyle='--')
plt.xlabel('Days', fontsize=13)
plt.ylabel('Close Price (USD)', fontsize=13)
plt.legend()
plt.tight_layout()
plt.savefig('prediction.png', dpi=150)
plt.show()

In [ ]:
model.save('model.h5')

from google.colab import files
files.download('model.h5')
files.download('closing_price.png')
files.download('prediction.png')